In [8]:
import hashlib
import rsa
import binascii
import os
from gmpy2 import mpz, iroot, powmod, mul, t_mod

In [9]:
def to_bytes(n):
    """ Return a bytes representation of a int """
    return n.to_bytes((n.bit_length() // 8) + 1, byteorder='big')

def from_bytes(b):
    """ Makes a int from a bytestring """
    return int.from_bytes(b, byteorder='big')

def get_bit(n, b):
    """ Returns the b-th rightmost bit of n """
    return ((1 << b) & n) >> b

def set_bit(n, b, x):
    """ Returns n with the b-th rightmost bit set to x """
    if x == 0: return ~(1 << b) & n
    if x == 1: return (1 << b) | n

def cube_root(n):
    return int(iroot(mpz(n), 3)[0])


In [10]:
message = "Ciao, mamma!!".encode("ASCII")
message_hash = hashlib.sha256(message).digest()

In [11]:
ASN1_blob = rsa.pkcs1.HASH_ASN1['SHA-256']
suffix = b'\x00' + ASN1_blob + message_hash

In [12]:
binascii.hexlify(suffix)

b'003031300d060960864801650304020105000420a39ddaae64be645e43d483713135d6fad7c25d2956abab8d9a3dfdcd2b1b821b'

In [13]:
len(suffix)

52

In [14]:
suffix[-1]&0x01 == 1 # easy suffix computation works only with odd target

True

In [15]:
sig_suffix = 1
for b in range(len(suffix)*8):
    if get_bit(sig_suffix ** 3, b) != get_bit(from_bytes(suffix), b):
        sig_suffix = set_bit(sig_suffix, b, 1)

In [16]:
to_bytes(sig_suffix ** 3).endswith(suffix) # BOOM

True

In [17]:
len(to_bytes(sig_suffix ** 3)) * 8

1248

In [18]:
while True:
    prefix = b'\x00\x01' + os.urandom(2048//8 - 2)
    sig_prefix = to_bytes(cube_root(from_bytes(prefix)))[:-len(suffix)] + b'\x00' * len(suffix)
    sig = sig_prefix[:-len(suffix)] + to_bytes(sig_suffix)
    if b'\x00' not in to_bytes(from_bytes(sig) ** 3)[:-len(suffix)]: break

In [19]:
to_bytes(from_bytes(sig) ** 3).endswith(suffix)

True

In [20]:
to_bytes(from_bytes(sig) ** 3).startswith(b'\x01')

True

In [21]:
len(to_bytes(from_bytes(sig) ** 3)) == 2048//8 - 1

True

In [22]:
b'\x00' not in to_bytes(from_bytes(sig) ** 3)[:-len(suffix)]

True

In [23]:
binascii.hexlify(sig)

b'2957a6db3b4d8a3f744f8a3b9e84a96cbb3a581c6b35d0fd9a88faf7353370807f7dc53e33e453b32c69e3caf4eb3e472ff0dfc29d65126b4f1eddd4a64bbd63ce192752903fee1fff985a48f1048993f91732a603'

In [24]:
binascii.hexlify(to_bytes(from_bytes(sig) ** 3))

b'0114062193ec44c073d39f898d7891edf515544ef23d95911cc1fcef22718b28b4749ce822fa1eb32ae46674a279762bfd60bfe666234f39a10231266b06fe91865c8b80169b906c90f9356c1116c9ed51d2a8191246572e2c458bbf0944cc2e7b0cfd511b749dae52e4115f93016728e1b7688c03cfae35930f5a390d4baa9c56d140098f35e53e14337a784021053670990cbeadde677531658fbe14a9febe0c6a572b31f5b17e9ce0fa76ca477ed7a9e89f18cc2ea5dcaea4feb39306e10b834451ee7fdea5546c4e3a003031300d060960864801650304020105000420a39ddaae64be645e43d483713135d6fad7c25d2956abab8d9a3dfdcd2b1b821b'

In [27]:
key = rsa.newkeys(2048)[0]
key.e = 3

In [28]:
rsa.verify(message, sig, key)

VerificationError: Verification failed